In [3]:
import os
import secrets
from PIL import Image
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField # BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError # Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta
import re

app = Flask(__name__)
# database
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)


login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    bio = db.Column(db.Text)
    admin = db.Column(db.Boolean)
    # image_file = db.Column(db.String(20), nullable=False, default='default.jpg')
    # notes = db.relationship('Note', backref='writer', lazy='dynamic')

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")    

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))

        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password,bio='',admin=0)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


















class zhoubian(db.Model):
    __tablename__ = 'zhoubian'
    # name = db.Column(db.text, primary_key=True)
    name = db.Column(db.Text,primary_key=True)
    label = db.Column(db.Text)
    price = db.Column(db.Text)
    pic_url = db.Column(db.Text)
    purchase_link = db.Column(db.Text)

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/youxiku")
def youxiku():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic ORDER BY game_rate')
    youxiku = []
    for i in sqlalchemyObj:
        youxiku.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in youxiku:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxiku.html', title='youxiku', dataset=dataset)

@app.route("/youxikucelue")
def youxikucelue():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic where game_label like "%策略%" ORDER BY game_rate')
    celue = []
    for i in sqlalchemyObj:
        celue.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in celue:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxikucelue.html', title='youxikucelue', dataset=dataset)

@app.route("/youxikuxiuxian")
def youxikuxiuxian():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic where game_label like "%休闲%" ORDER BY game_rate')
    xiuxian = []
    for i in sqlalchemyObj:
        xiuxian.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in xiuxian:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxikuxiuxian.html', title='youxikuxiuxian', dataset=dataset)

@app.route("/youxikumaoxian")
def youxikumaoxian():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic where game_label like "%冒险%" ORDER BY game_rate')
    maoxian = []
    for i in sqlalchemyObj:
        maoxian.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in maoxian:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxikumaoxian.html', title='youxikumaoxian', dataset=dataset)

@app.route("/youxikujiemi")
def youxikujiemi():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic where game_label like "%解谜%" ORDER BY game_rate')
    jiemi = []
    for i in sqlalchemyObj:
        jiemi.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in jiemi:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxikujiemi.html', title='youxikujiemi', dataset=dataset)

@app.route("/youxikubanyan")
def youxikubanyan():
    sqlalchemyObj = db.engine.execute('select game_pic, game_name, game_price, detail_url, game_label from game_basic where game_label like "%角色扮演%" ORDER BY game_rate')
    banyan = []
    for i in sqlalchemyObj:
        banyan.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in banyan:
        dict['game_pic'] = i[0]
        dict['game_name'] = i[1]
        dict['game_price'] = i[2]
        dict['detail_url'] = i[3]
        dict['game_label'] = i[4]
        
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('youxikubanyan.html', title='youxikubanyan', dataset=dataset)

@app.route("/youxi1")
def youxi1():
    return render_template('youxi1.html')

@app.route("/youxi_loading")
def youxi1_loading():
    return render_template('youxi_loading.html')

@app.route("/youxi1_dsqjh")
def youxi1_dsqjh():
    return render_template('youxi1_dsqjh.html')

@app.route("/youxi1_gwzj")
def youxi1_gwzj():
    return render_template('youxi1_gwzj.html')

@app.route("/youxi1_zrcs")
def youxi1_zrcs():
    return render_template('youxi1_zrcs.html')

@app.route("/youxi1_gjj")
def youxi1_gjj():
    return render_template('youxi1_gjj.html')

@app.route("/youxi1_wmdyt")
def youxi1_wmdyt():
    return render_template('youxi1_wmdyt.html')

@app.route("/tuandui")
def tuandui():

    sqlalchemyObj = db.engine.execute('select pic_url,inherent_url,title,source,author from review ORDER BY title')
    tuandui = []
    for i in sqlalchemyObj:
        tuandui.append(i)
    dataset = []
    dict={}
    for i in tuandui:
        dict['pic_url'] = i[0]
        dict['inherent_url'] = i[1]
        dict['title'] = i[2]
        dict['source'] = i[3]
        dict['author'] = i[4]
    
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('tuandui.html', title='tuandui', dataset=dataset)

@app.route("/tuandui1")
def tuandui1():
    return render_template('tuandui1.html')

@app.route("/tuandui2")
def tuandui2():
    return render_template('tuandui2.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route("/zhoubian")
def zhoubian():

    sqlalchemyObj = db.engine.execute('select * from zhoubian ORDER BY label')
    zhoubian = []
    for i in sqlalchemyObj:
        zhoubian.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in zhoubian:
        dict['label'] = i[0]
        dict['name'] = i[1]
        dict['price'] = i[4]
        dict['pic_url'] = i[2]
        dict['purchase_link'] = i[3]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('zhoubian.html', title='zhoubian', dataset=dataset)

@app.route("/zhoubian_dianzi")
def zhoubian_dianzi():

    sqlalchemyObj = db.engine.execute('select * from zhoubian where label="电子周边" ORDER BY name')
    zhoubian_dianzi = []
    for i in sqlalchemyObj:
        zhoubian_dianzi.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in zhoubian_dianzi:
        dict['label'] = i[0]
        dict['name'] = i[1]
        dict['price'] = i[4]
        dict['pic_url'] = i[2]
        dict['purchase_link'] = i[3]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('zhoubian_dianzi.html', title='zhoubian_dianzi', dataset=dataset)

@app.route("/zhoubian_shiti")
def zhoubian_shiti():

    sqlalchemyObj = db.engine.execute('select * from zhoubian where label="实体周边" ORDER BY name')
    zhoubian_shiti = []
    for i in sqlalchemyObj:
        zhoubian_shiti.append(i)
    # print(zhoubian)
    dataset = []
    dict={}
    for i in zhoubian_shiti:
        dict['label'] = i[0]
        dict['name'] = i[1]
        dict['price'] = i[4]
        dict['pic_url'] = i[2]
        dict['purchase_link'] = i[3]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('zhoubian_shiti.html', title='zhoubian_shiti', dataset=dataset)

@app.route("/zhoubian1")
def zhoubian1():
    return render_template('zhoubian1')

################################################################################################################

# ERROR HANDLERS

################################################################################################################

# @app.errorhandler(404)
# def page_not_found(e):
#     # note that we set the 404 status explicitly
#     return "Error 404 - page not found."

################################################################################################################

# APPLICATION TEST RUN AT PORT 9001

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 9030)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\ProgramData\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9030/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 07:11:45] "GET /property-single.html HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 07:11:46] "GET /property-single.html HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 07:11:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 07:11:50] "GET /youxi1_gjj HTTP/1.1" 200 -
